# Ingest gene expression data from TCGA, TARGET and GTEX

Download gene expression data from the UCSC Xena Toil re-compute dataset. This dataset comprises gene expression data for 19 thousand samples all processed using the same genomics pipeline and therefore comparable to each other. Each sample is a vector of the floating point expression relative level for each of 60k genes. Associated with these data is clinical information on each sample such as type (tumor vs. normal), primary site (where the sample came from) etc... We use this information to split the dataset into one-hot tumor vs. normal as well as into a set of classes based on primary site for stratification.  

In [8]:
import os
import requests
import numpy as np
import pandas as pd
import h5py

In [11]:
%%time
"""
Download the expression data from xena, if we haven't already, and save in 
an hdf5 file. This can take around 30 minutes between the download and
the conversion from tsv into float32 dataframes so we save the tsv and hdf5
file for future use. We download manually vs. passing read_csv a url 
directly as the latter times out with this size file. Note we invert
the expression matrix to conform to machine learning where the samples
are in rows vs. the standard genomics world where samples columns
and reatures (i.e. genes) are rows.
"""
if not os.path.exists("data/TcgaTargetGtex_rsem_gene_tpm"):
    print("Downloading TCGA, TARGET and GTEX expression data from UCSC Xena")
    with open("data/TcgaTargetGtex_rsem_gene_tpm", "wb") as f:
        r = requests.get("https://toil.xenahubs.net/download/TcgaTargetGtex_rsem_gene_tpm", stream=True)
        r.raise_for_status()
        for block in r.iter_content(32768):
            f.write(block)

if not os.path.exists("data/TcgaTargetGtex_rsem_gene_tpm.hd5"):
    print("Converting expression to dataframe and storing in hdf5 file")
    expression = pd.read_csv("data/TcgaTargetGtex_rsem_gene_tpm", 
                             sep="\t", index_col=0).dropna().astype(np.float32).T
    expression.to_hdf("data/TcgaTargetGtex_rsem_gene_tpm.hd5", "expression", mode="w", format="fixed")

X = pd.read_hdf("data/TcgaTargetGtex_rsem_gene_tpm.hd5", "expression").sort_index(axis=0)

CPU times: user 8.99 s, sys: 46 s, total: 55 s
Wall time: 55.2 s


In [12]:
X.head()

sample,ENSG00000242268.2,ENSG00000259041.1,ENSG00000270112.3,ENSG00000167578.16,ENSG00000278814.1,ENSG00000078237.5,ENSG00000269416.5,ENSG00000263642.1,ENSG00000146083.11,ENSG00000158486.13,...,ENSG00000009694.13,ENSG00000238244.3,ENSG00000216352.1,ENSG00000123685.8,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.18,ENSG00000231119.2,ENSG00000280861.1,ENSG00000181518.3
GTEX-1117F-0226-SM-5GZZ7,-9.9658,-9.9658,-4.2934,5.1190,-9.9658,0.8488,-9.9658,-9.9658,5.1498,-9.9658,...,2.4571,-1.4305,-9.9658,2.8178,-0.1031,-9.9658,5.1631,-3.3076,-9.9658,-9.9658
GTEX-1117F-0426-SM-5EGHI,-9.9658,-9.9658,0.0014,4.1277,-9.9658,0.6880,-9.9658,-9.9658,3.4830,-9.9658,...,-0.9132,-9.9658,-9.9658,-0.9406,-9.9658,-1.5105,4.1764,-5.0116,-9.9658,-9.9658
GTEX-1117F-0526-SM-5EGHJ,-9.9658,-9.9658,-9.9658,4.4067,-9.9658,0.0440,-9.9658,-9.9658,4.3841,-9.9658,...,1.5165,-9.9658,-9.9658,1.7141,-1.1488,-9.9658,4.8768,-9.9658,-9.9658,-9.9658
GTEX-1117F-0626-SM-5N9CS,-1.2481,-9.9658,-5.5735,5.6860,-9.9658,1.3679,-9.9658,-9.9658,5.0644,-1.8836,...,1.5998,-9.9658,-9.9658,3.9356,-1.1488,-1.0559,4.8694,-1.9379,-9.9658,-9.9658
GTEX-1117F-0726-SM-5GIEN,-3.8160,-9.9658,0.3573,4.0357,-9.9658,-0.4325,-5.5735,-9.9658,3.9421,-3.4580,...,-3.1714,-9.9658,-9.9658,0.6608,-1.1811,-9.9658,3.6816,-2.6349,-9.9658,-9.9658


In [13]:
# Read in the clinical (i.e. labels) from xena
Y = pd.read_table("https://toil.xenahubs.net/download/TcgaTargetGTEX_phenotype.txt",
                  header=0, names=["detailed_category", "primary_site", "sample_type", "gender", "study"],
                  sep="\t", encoding="ISO-8859-1", index_col=0, dtype="str").sort_index(axis=0)

# Compute and add a tumor/normal column
Y["tumor_normal"] = Y.apply(
    lambda row: "Normal" if row["sample_type"] in ["Cell Line", "Normal Tissue", "Solid Tissue Normal"]
    else "Tumor", axis=1)

Y.head()

,detailed_category,primary_site,sample_type,gender,study,tumor_normal
GTEX-1117F-0226-SM-5GZZ7,Adipose - Subcutaneous,Adipose Tissue,Normal Tissue,Female,GTEX,Normal
GTEX-1117F-0426-SM-5EGHI,Muscle - Skeletal,Muscle,Normal Tissue,Female,GTEX,Normal
GTEX-1117F-0526-SM-5EGHJ,Artery - Tibial,Blood Vessel,Normal Tissue,Female,GTEX,Normal
GTEX-1117F-0626-SM-5N9CS,Artery - Coronary,Blood Vessel,Normal Tissue,Female,GTEX,Normal
GTEX-1117F-0726-SM-5GIEN,Heart - Atrial Appendage,Heart,Normal Tissue,Female,GTEX,Normal


In [14]:
Y.describe()

,detailed_category,primary_site,sample_type,gender,study,tumor_normal
count,19130,19126,19131,18972,19131,19131
unique,93,46,17,2,3,2
top,Breast Invasive Carcinoma,Brain,Primary Tumor,Male,TCGA,Tumor
freq,1212,1846,9185,10456,10535,10531


In [16]:
# Remove rows where the label is null or the sample is missing
Y_not_null = Y[pd.notnull(Y["primary_site"])]
intersection = X.index.intersection(Y_not_null.index)
X_clean = X[X.index.isin(intersection)]
Y_clean = Y[Y.index.isin(intersection)]

# Make sure the label and example samples are in the same order
assert(X_clean.index.equals(Y_clean.index))

print(intersection.shape[0], "samples with non-null labels")

19126 samples with non-null labels


In [17]:
# Convert tumor/normal labels to binary 1/0
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_binary = encoder.fit_transform(Y_clean["tumor_normal"])

In [21]:
%%time
# Split into stratified training and test sets based on classes (i.e. tissue type)
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(X_clean.values, Y_clean["primary_site"]):
    X_train, X_test = X.values[train_index], X_clean.values[test_index]
    y_train, y_test = y_binary[train_index], y_binary[test_index]

CPU times: user 16.1 s, sys: 11 s, total: 27.1 s
Wall time: 26.9 s


In [ ]:
# Convert from ensembl to hugo gene names as the latter are more commonly known
# ensembl = pd.read_table("ftp://ftp.ebi.ac.uk/pub/databases/genenames/new/tsv/hgnc_complete_set.txt", 
#                         usecols=["symbol", "ensembl_gene_id", "gene_family"],
#                         index_col=2)
# ensembl.head()
# ensembl.loc["ENSG00000121410"].symbol

In [24]:
"""
Write to an h5 file for training
X: examples as a numpy floating point array
y: binary to predict
features: string array of the features in X (Ensemble gene ids)
labels: array of strings coresponding to the binary labels
"""
with h5py.File("data/tumor_normal.h5", "w") as f:
    f.create_dataset('X_train', X_train.shape, dtype='f')[:] = X_train
    f.create_dataset('X_test', X_test.shape, dtype='f')[:] = X_test
    f.create_dataset('y_train', y_train.shape, dtype='i')[:] = y_train
    f.create_dataset('y_test', y_test.shape, dtype='i')[:] = y_test
    f.create_dataset('features', X_clean.columns.shape, 'S10', 
                     [l.encode("ascii", "ignore") for l in X_clean.columns.values])
    f.create_dataset('labels', (2, 1), 'S10', 
                     [l.encode("ascii", "ignore") for l in ["Normal", "Tumor"]])